In [1]:
import jsonc

In [2]:
original = "loottables_original.json"

In [23]:
with open(original, "r") as file:
    loottable_dict = jsonc.load(file)

In [6]:
for key in loottable_dict:
    print(key)

MagicEffectsCount
RestrictedItems
ItemSets
LootTables


## MagicEffectsCount Functions
### Functions that change the ammount, and it's associated probability, of enchantments

In [4]:
def increase_enchant_count(n):
    """
    Increases all magic effects count by n.
    Changes probabilities only indirectly.
    """
    print("########################## BEFORE ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
        for i in range(len(probabilities)):
            probabilities[i][0] += 1
    
    print("########################## AFTER ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
    
    print("############################################################")
    print("##################### INCREASE COUNT #######################")
    print("############################################################")
    print()
    print()
    return 1

def modify_probabilities(p):
    """
    Takes p away from the first entry and adds 2*p/3 and p/3 to the second and last entry, respectively
    """
    print("########################## BEFORE ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
        probabilities[0][1] -= p
        probabilities[1][1] += (p//3) * 2
        probabilities[2][1] = 100 - probabilities[0][1] - probabilities[1][1]
    
    print("########################## AFTER ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
    
    print("############################################################")
    print("################### MODIFY PROBABILITIES ###################")
    print("############################################################")
    print()
    print()
    return 1

## ItemSets Functions
### Functions that create/delete item sets

###### Note: EpicLoot provides a setting to determine the item-to-material drop ratio. The only provided function to change item sets in this code aims to improve configuration in case you are running CLLC and want only materials dropping. In such cases, utilising solely the EpicLoot configuration, I was met with bosses trophies drop not scaling with starts. The solution was to manually change the loottables.json file.

In [5]:
def only_enchanting_mats():
    """
    Deletes all item sets that don't include enchanting mats
    """
    item_set_index = 0
    while item_set_index < len(loottable_dict["ItemSets"]):
        item_set = loottable_dict["ItemSets"][item_set_index]
        #print(item_set["Name"], [char_seq in item_set["Name"] for char_seq in ["Mats", "Runestone"]])
        if not any([char_seq in item_set["Name"] for char_seq in ["Mats", "Runestone"]]):
            print(item_set["Name"], "Deleted")
            loottable_dict["ItemSets"].pop(item_set_index)
        else:
            item_set_index += 1

## Non-vanilla mobs

In [31]:
def add_mob_objects():
    """
    Creates mob objects for all mob ids added in the mobs_additions.txt file, each with their
    respective tier, also defined in the file
    """
    with open("mobs_additions.txt", "r") as mobs_file:
        for line in mobs_file:
            PRESENT = False
            if line[0] == "#" or line == "\n":
                continue
            mob_id, tier = line.replace("\n", "").split()
            for obj in  loottable_dict["LootTables"]:
                if mob_id == obj["Object"]:
                    PRESENT = True
                    break
            if PRESENT:
                print(f"{mob_id}{(20-len(mob_id)) * ' '}\t{tier}" + f"{10 * ' '}" + "ALREADY ADDED")
                continue
            print(f"{mob_id}{(20-len(mob_id)) * ' '}\t{tier}", end=" ")
            mob_obj = {}
            mob_obj["Object"] = mob_id
            mob_obj["RefObject"] = f"Tier{tier}Mob"
            loottable_dict["LootTables"].insert(8, mob_obj)
            print(mob_obj, f"{(65-len(str(mob_obj))) * ' '}ADDED")

In [32]:
add_mob_objects()

Fox_TW              	0          ALREADY ADDED
Razorback_TW        	0          ALREADY ADDED
BlackBear_TW        	1          ALREADY ADDED
GDAncientShaman_TW  	1          ALREADY ADDED
Crawler_TW          	2          ALREADY ADDED
HelWraith_TW        	2          ALREADY ADDED
RottingElk_TW       	3          ALREADY ADDED
GrizzlyBear_TW      	3          ALREADY ADDED
ObsidianGolem_TW    	4          ALREADY ADDED
Prowler_TW          	4          ALREADY ADDED


## Generate Mob Loot Tables
### Functions to generate the loot tables for mob tiers and bosses

In [37]:
for b in loottable_dict["LootTables"][:1]:
    print(b["LeveledLoot"])

[{'Level': 1, 'Drops': [[0, 100]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [100, 0, 0, 0]}]}, {'Level': 2, 'Drops': [[0, 95], [1, 5]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [95, 5, 0, 0]}]}, {'Level': 3, 'Drops': [[0, 90], [1, 10]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [90, 10, 0, 0]}]}, {'Level': 4, 'Drops': [[0, 80], [1, 20]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [80, 15, 5, 0]}]}, {'Level': 5, 'Drops': [[0, 70], [1, 30]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [70, 19, 10, 1]}]}, {'Level': 6, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}, {'Level': 7, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}, {'Level': 8, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}]


In [40]:
tiers = set()
for obj in loottable_dict["LootTables"]:
    if "RefObject" in obj.keys():
        tiers.add(obj["RefObject"])
tiers = list(tiers)
tiers.sort()
print(tiers)

['Tier0Mob', 'Tier1Mob', 'Tier2Mob', 'Tier3Mob', 'Tier4Mob', 'Tier5Mob', 'Tier6Mob', 'Tier7Mob']


In [294]:
import numpy as np

def aux_calc(level, tier):
    if tier == 0:
        tier += 1
    return int(np.log2(tier)*1.4) + int(np.log2(level)*1.9) + 1

def prob_calc(total_prob, tier, level, count):
    try:
        ret_val = pow(tier+1, 0.1) * abs ( total_prob // (level) )
        return int ( ret_val )
    except ZeroDivisionError:
        ret_val = pow(tier+1, 0.1) * abs ( total_prob // (level + 1) )
        return int ( ret_val )


def generate_mobs_loot_tables(max_star):
    """
    Generate the loot tables for each mob tier based on the maximum starts fro mobs
    """
    for tier in tiers:
        tier_num = int(tier[4])
        print(f"TIER {tier}")
        leveled_loot = []
        for level in range(1, max_star+1):
            drops = []
            loot = []
            print("level", level, end=": ")
            start_drop_count = max(0, aux_calc(level, tier_num)-2)
            stop_drop_count = aux_calc(level, tier_num)+1
            total_probability = 100
            i = 0
            for drop_count in range(start_drop_count, stop_drop_count):      
                probability = prob_calc(total_probability, tier_num, level, drop_count - start_drop_count)
                total_probability -= probability
                if total_probability < 0:
                    probability += total_probability
                drops.append([drop_count, probability])
                print(f"{drops[i]}", end=" ")
                i += 1
            print(f"<{total_probability}>", end="    ")
            print()
        print()

In [295]:
generate_mobs_loot_tables(8)

TIER Tier0Mob
level 1: [0, 100] [1, 0] <0>    
level 2: [0, 50] [1, 25] [2, 12] <13>    
level 3: [2, 33] [3, 22] [4, 15] <30>    
level 4: [2, 25] [3, 18] [4, 14] <43>    
level 5: [3, 20] [4, 16] [5, 12] <52>    
level 6: [3, 16] [4, 14] [5, 11] <59>    
level 7: [4, 14] [5, 12] [6, 10] <64>    
level 8: [4, 12] [5, 11] [6, 9] <68>    

TIER Tier1Mob
level 1: [0, 100] [1, -7] <-14>    
level 2: [0, 53] [1, 24] [2, 11] <12>    
level 3: [2, 35] [3, 22] [4, 15] <28>    
level 4: [2, 26] [3, 19] [4, 13] <42>    
level 5: [3, 21] [4, 16] [5, 12] <51>    
level 6: [3, 17] [4, 13] [5, 11] <59>    
level 7: [4, 15] [5, 12] [6, 10] <63>    
level 8: [4, 12] [5, 11] [6, 9] <68>    

TIER Tier2Mob
level 1: [0, 100] [1, -11] [2, -23] <-48>    
level 2: [1, 55] [2, 24] [3, 11] <10>    
level 3: [3, 36] [4, 23] [5, 14] <27>    
level 4: [3, 27] [4, 20] [5, 14] <39>    
level 5: [4, 22] [5, 16] [6, 13] <49>    
level 6: [4, 17] [5, 14] [6, 12] <57>    
level 7: [5, 15] [6, 13] [7, 11] <61>    
lev

In [289]:
def prob_calc(total_prob, tier, level, count):
    try:
        return abs ( total_prob // (15 - level + count) )
    except ZeroDivisionError:
        return abs ( total_prob // (14 - level + count) )
    
    
tier_num = int(tiers[1][4])
for level in range(1, 9):
    print("level", level, end=": ")
    remaining_prob = 100
    start_drop_count = max(0, aux_calc(level, tier_num)-2)
    stop_drop_count = aux_calc(level, tier_num)+1
    for drop_count in range(start_drop_count, stop_drop_count):
        #prob = remaining_prob // ( tier_num + level )
        prob = prob_calc(remaining_prob, tier_num, level, drop_count - start_drop_count)
        remaining_prob -= prob
        print(f"{drop_count} ({prob})", end="  ")
    print(f"<{remaining_prob}>")

level 1: 0 (7)  1 (6)  <87>
level 2: 0 (7)  1 (6)  2 (5)  <82>
level 3: 2 (8)  3 (7)  4 (6)  <79>
level 4: 2 (9)  3 (7)  4 (6)  <78>
level 5: 3 (10)  4 (8)  5 (6)  <76>
level 6: 3 (11)  4 (8)  5 (7)  <74>
level 7: 4 (12)  5 (9)  6 (7)  <72>
level 8: 4 (14)  5 (10)  6 (8)  <68>


In [249]:
len("level 7: [6, 10] [7, 10] [8, 10] <70>    ")

41

In [ ]:
mod